In [5]:
#%% 
# =============================================================================
# validation 
# =============================================================================
import requests
import json
from bs4 import BeautifulSoup as BS
from IPython.display import display, Image
#'https://api.themoviedb.org/3/movie/550?api_key='
## extract id
response = requests.get('http://www.imdb.com/find?q=Toy%20Story(1995)')
soup = BS(response.text,"lxml")
id_href = soup.select('.result_text')[0].a['href']
print(id_href.split('/')[-2]) # extract id--imdb

## get baseurl structure, w185 corresponds to size of movie poster.
## query themoviedb api (https://www.themoviedb.org)
headers = {'Accept': 'application/json'}
payload = {'api_key': 'cf8a234e722de1a4f390ce5b62c5b327'} 
response = requests.get("http://api.themoviedb.org/3/configuration", params=payload, headers=headers)
response = response.json()
base_url = response['images']['base_url'] + 'w185'


# %%

def get_poster(imdb_url, base_url):
    # Get IMDB movie ID
    response = requests.get(imdb_url)
    soup = BS(response.text,"lxml")
    id_href = soup.select('.result_text')[0].a['href']    
    movie_id = id_href.split('/')[-2]
    
    # Query themoviedb.org API for movie poster path.
    movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(movie_id)
    headers = {'Accept': 'application/json'}
    payload = {'api_key': 'cf8a234e722de1a4f390ce5b62c5b327'} 
    response = requests.get(movie_url, 
                            params=payload, 
                            headers=headers)
    try:
        file_path = response.json()['posters'][0]['file_path']
    except:
        # IMDB movie ID is sometimes no good. Need to get correct one.
        movie_title = imdb_url.split('?')[-1].split('(')[0]
        payload['query'] = movie_title
        response = requests.get('http://api.themoviedb.org/3/search/movie', params=payload, headers=headers)
        movie_id = json.loads(response.text)['results'][0]['id']
        payload.pop('query', None)
        movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(movie_id)
        response = requests.get(movie_url, params=payload, headers=headers)
        file_path = json.loads(response.text)['posters'][0]['file_path']
        
    return base_url + file_path


toy_story = 'http://www.imdb.com/find?q=Toy%20Story(1995)'
imag_url = get_poster(toy_story,base_url)
img = Image(url = imag_url)

tt0114709


In [6]:
display(img)